In [8]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
import mysql.connector
import ssl

# konektor ke database
db = mysql.connector.connect(
        host='localhost',
        user='root',
        password='',
        database='UnNgGrape'
        )
cursor = db.cursor()
exist = 0
add = 0

# scrape berdasarkan data keyword
cursor.execute("SELECT * FROM keyword")
raw_keyword = cursor.fetchall()
for row2 in raw_keyword:
    # penyiapan scraping
    keyword = row2[0]
    try:
        r = Request("https://glints.com/id/opportunities/jobs/explore?keyword=" + keyword +"&country=ID&locationName=Indonesia", headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})
        gcontext = ssl.SSLContext()  # Only for gangstars
        response = urlopen(r, context=gcontext).read()
        soup = BeautifulSoup(response, "lxml")
    except:
        continue

    # inisialisasi array untuk menampilkan hasil
    title_lowongan = []
    nama_perusahaan = []
    lokasi_perusahaan = []
    link_lowongan = []

    jobList = soup.find_all("div","compact_job_card")
    for p in jobList:
        link = "https://glints.com"+p.find('a').get('href')
        
        # mengecek apakah data sudah adat atau belum
        cursor.execute("SELECT * FROM lowongan WHERE link_lowongan = '" + link + "'")
        data = cursor.fetchall()
        # if else untuk pengecekan input data
        if data:
            exist= exist + 1
        else:
            asal = "glints"

            lowongan = p.find("h3", "CompactOpportunityCardsc__JobTitle-sc-1xtox99-7 kJpKeD").get_text()
            # menggunakan try except karena ada beberapa perusahaan yang dirahasiakan
            try:
                perusahaan = p.find('a', "CompactOpportunityCardsc__CompanyLink-sc-1xtox99-8 bFxfxR").get_text()
            except:
                perusahaan = "Perusahaan Dirahasiakan"
            lokasi = p.find("div", "CompactOpportunityCardsc__OpportunityInfo-sc-1xtox99-13 hiAtMr").get_text()

            cursor.execute(
                        "INSERT INTO lowongan(asal, title_lowongan, nama_perusahaan, lokasi_perusahaan, link_lowongan)"
                        "VALUES ('"+ asal +"', '"+ lowongan.replace("'", "").replace('"', '').replace(',', '') +"', '"+ perusahaan.replace(',', '').replace("'", "").replace('"', '') +"', '"+ lokasi.replace("'", "").replace('"', '').replace(',', '') +"', '"+ link +"')"
                        )

            db.commit()
            add = add + 1
            
            title_lowongan.append(lowongan)
            nama_perusahaan.append(perusahaan)
            lokasi_perusahaan.append(lokasi)
            link_lowongan.append(link)
            
print("Data exists: " + str(exist))
print("Data has been added: " + str(add))
jobList_dict ={'lowongan':title_lowongan, 'perusahaan':nama_perusahaan, 'lokasi':lokasi_perusahaan, 'link':link_lowongan}
df = pd.DataFrame(jobList_dict,columns = ['lowongan','perusahaan', 'lokasi', 'link'])

df.sort_values('lowongan',ascending=True)

Data exists: 2599
Data has been added: 11


,lowongan,perusahaan,lokasi,link
